## Data Importation, Cleaning, and Transformation

Data Source: https://eodhistoricaldata.com/

### Import Dependencies

In [1]:
# import dependencies
import numpy as np
import scipy as sp
import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib as jl
import datetime as dt
import requests

from icecream import ic

import os
from dotenv import load_dotenv

from ta import add_all_ta_features
from ta.trend import MACD
from ta.volatility import BollingerBands
from ta.volume import VolumeWeightedAveragePrice
from ta.momentum import StochRSIIndicator

In [2]:
# load API keys from .env file
load_dotenv()
TRADIER_TOKEN = os.getenv('TRADIER_TOKEN')
EOD_TOKEN = os.getenv('EOD_TOKEN')

In [3]:
df = pd.read_csv('data/etf_list.csv')

### Get Tickers

In [4]:
def get_etf_tickers(): 
    
    # pulls all tickers of ETFs on NYSE or NASDAQ

    r = requests.get('https://eodhistoricaldata.com/api/exchange-symbol-list/US', 
        params={'api_token': EOD_TOKEN, 'fmt': 'json'}
        )
    data = r.json()
    r.close()

    df = pd.DataFrame(data)
    df = df[
        (df.Type == 'ETF') &
        ((df.Exchange == 'NYSE ARCA') |
        (df.Exchange == 'NASDAQ'))
        ]

    df.index = df.Code
    df.drop('Code', axis = 1, inplace=True)
    ticker_list = list(df.index)
    return ticker_list

In [235]:
# get etf tickers
# tickers = get_etf_tickers()

In [5]:
# save ticker list to csv
# df = pd.DataFrame(tickers)
# df.to_csv('data/tickers.csv')

### Get Historical Price

In [101]:
def get_historical_price(tickers, data_type):

    # pulls historical daily or intraday OLHC prices and volume

    d = {}

    for i in range(len(tickers)): 

        r = requests.get('https://eodhistoricaldata.com/api' + '/' + data_type + '/' + tickers[i] + '.US', 
            params={'api_token': EOD_TOKEN, 'fmt': 'json'}
            )
        data = r.json()
        r.close()

        d[tickers[i]] = data

        # ADD PROGRESS BAR
        
        # remove unecessary loop
        d[tickers[i]] = pd.DataFrame.from_records(data)

        indicator_macd = MACD(close = d[tickers[i]]['close'], window_slow=26, window_fast=12, window_sign=9, fillna=True)
        d[tickers[i]]['macd'] = indicator_macd.macd()
        d[tickers[i]]['macd_diff'] = indicator_macd.macd_diff()
        d[tickers[i]]['macd_signal'] = indicator_macd.macd_signal()

        indicator_bb = BollingerBands(close = d[tickers[i]]['close'], window=20, window_dev=2, fillna=True)
        d[tickers[i]]['bb_mavg'] = indicator_bb.bollinger_mavg()
        d[tickers[i]]['bb_hband'] = indicator_bb.bollinger_hband()
        d[tickers[i]]['bb_lband'] = indicator_bb.bollinger_lband()
        d[tickers[i]]['bb_hband_ind'] = indicator_bb.bollinger_hband_indicator()
        d[tickers[i]]['bb_lband_ind'] = indicator_bb.bollinger_lband_indicator()

        indicator_vwap = VolumeWeightedAveragePrice(
            high = d[tickers[i]]['high'],
            low = d[tickers[i]]['low'],
            close = d[tickers[i]]['close'],
            volume = d[tickers[i]]['volume'],
            window=14, fillna=True)
        d[tickers[i]]['vwap'] = indicator_vwap.volume_weighted_average_price()

        indicator_stochrsi = StochRSIIndicator(close = d[tickers[i]]['close'], window=14, smooth1=3, smooth2=3, fillna=True)
        d[tickers[i]]['stoch_rsi'] = indicator_stochrsi.stochrsi()
        d[tickers[i]]['stochrsi_d'] = indicator_stochrsi.stochrsi_d()
        d[tickers[i]]['stochrsi_k'] = indicator_stochrsi.stochrsi_k()

    return d

In [103]:
# placeholder tickers
tickers = ['SPXS', 'SPXL']

# get intraday prices | dictionary of dataframes
intraday_data = get_historical_price(tickers, 'intraday')

# get daily prices | dictionary of dataframes
daily_data = get_historical_price(tickers, 'eod')

In [104]:
# concat intraday dataframes together and convert to dask df
intraday_df = pd.concat(intraday_data.values(), axis=1, keys=intraday_data.keys())
intraday_dask_df = dd.from_pandas(intraday_df, npartitions=6)

# concat daily dataframes together and convert to dask df
daily_df = pd.concat(daily_data.values(), axis=1, keys=daily_data.keys())
daily_dask_df = dd.from_pandas(daily_df, npartitions=6)

#daily_df.to_csv('data/test_daily_df.csv')

In [154]:
daily_df['SPXL'].adjusted_close
#daily_df['SPXS'].adjusted_close

0        3.3851
1        2.8673
2        3.0698
3        2.9503
4        2.7549
         ...   
3584    79.0300
3585    76.5300
3586    75.1500
3587    78.0000
3588    75.5000
Name: adjusted_close, Length: 3589, dtype: float64

### Get ETF Fundementals

In [204]:
def get_etf_fundementals(tickers): 

    # pulls fundementals and wrangles data into multiple dfs

    raw_data = {}
    #single_ticker_clean_data = {}
    all_clean_data = {}


    columns = ['ISIN', 'Company_Name', 'Company_URL', 'ETF_URL', 'Domicile',
        'Index_Name', 'Yield', 'Dividend_Paying_Frequency', 'Inception_Date',
        'Max_Annual_Mgmt_Charge', 'Ongoing_Charge', 'Date_Ongoing_Charge',
        'NetExpenseRatio', 'AnnualHoldingsTurnover', 'TotalAssets', 'Holdings_Count',
        'Average_Mkt_Cap_Mil']

    for i in range(len(tickers)): 

        single_ticker_clean_data = {}
        
        r = requests.get('https://eodhistoricaldata.com/api/fundamentals/' + tickers[i] + '.US', 
            params={'api_token': '63dc0e2f4efc43.34327983', 'fmt': 'json'}
            )
        data = r.json()
        
        r.close()

        # place raw data for each ticker inside nested dict
        raw_data[tickers[i]] = data    

        a = pd.Series(raw_data[tickers[i]]['General'])

        b = pd.Series([raw_data[tickers[i]]['ETF_Data'][name] for name in columns],
            index=[name for name in columns])

        # figure out fix
        # c = pd.DataFrame([d[tickers[i]]['ETF_Data']['Market_Capitalisation']])#,
        # index = [d[tickers[i]]['ETF_Data']['Market_Capitalisation'].keys()])
        # c = pd.Series(d[symbol]['ETF_Data']['Market_Capitalisation'],
        #  index = ['Market_Capitalisation'])


        c = pd.Series(raw_data[tickers[i]]['ETF_Data']['MorningStar'])

        # figure out fix
        single_ticker_clean_data['Performance'] = pd.DataFrame(
            raw_data[tickers[i]]['ETF_Data']['Performance'].items(),
            index = raw_data[tickers[i]]['ETF_Data']['Performance'].keys()).drop(0, axis=1)
        
        single_ticker_clean_data['Technicals'] = pd.Series(
            raw_data[tickers[i]]['Technicals'])

        single_ticker_clean_data['general'] = pd.DataFrame(pd.concat([a, b, c]),
            columns = ['data'],
            )
        single_ticker_clean_data['asset_allocation'] = pd.DataFrame(raw_data[tickers[i]]['ETF_Data']['Asset_Allocation'])
        # fix this
        region_weights_df = pd.DataFrame(raw_data[tickers[i]]['ETF_Data']['World_Regions'])
        sector_weights_df = pd.DataFrame(raw_data[tickers[i]]['ETF_Data']['Sector_Weights'])
        single_ticker_clean_data['fixed_income'] = pd.DataFrame(raw_data[tickers[i]]['ETF_Data']['Fixed_Income'])
        single_ticker_clean_data['top_10_holdings'] = pd.DataFrame(raw_data[tickers[i]]['ETF_Data']['Top_10_Holdings'].values(), index = raw_data[tickers[i]]['ETF_Data']['Top_10_Holdings'].keys())
        single_ticker_clean_data['holdings'] = pd.DataFrame(raw_data[tickers[i]]['ETF_Data']['Holdings'].values(), index = raw_data[tickers[i]]['ETF_Data']['Holdings'].keys())
        single_ticker_clean_data['valuations_growth'] = pd.DataFrame(raw_data[tickers[i]]['ETF_Data']['Valuations_Growth']).T

        # fix this
        placeholder = {}
        placeholder['region_weights'] = region_weights_df
        placeholder['sector_weights'] = sector_weights_df
        weights_df = pd.concat(placeholder.values(), axis=1, keys=placeholder.keys())
        single_ticker_clean_data['weights'] = weights_df.copy()

        all_clean_data[tickers[i]] = single_ticker_clean_data


    return all_clean_data

In [231]:
# # placeholder tickers
tickers = ['SPXL', 'SPXS']

# # get fundemental data
etf_fundemental_data = get_etf_fundementals(tickers)

In [234]:
etf_fundemental_data['SPXL'].keys()

dict_keys(['Performance', 'Technicals', 'general', 'asset_allocation', 'fixed_income', 'top_10_holdings', 'holdings', 'valuations_growth', 'weights'])

### Get Stock Fundementals

In [224]:
def get_stock_fundmentals(tickers): 

    raw_data = {}
    all_clean_data = {}

    columns = ['General', 'Highlights']

    for i in range(len(tickers)): 

        single_ticker_clean_data = {}

        r = requests.get('https://eodhistoricaldata.com/api/fundamentals/' + tickers[i] + '.US', 
            params={'api_token': '63dc0e2f4efc43.34327983', 'fmt': 'json'}
            )
        data = r.json()
        
        r.close()

        raw_data[tickers[i]] = data
        
        Officers = raw_data[tickers[i]]['General'].pop('Officers', None)
        Listings = raw_data[tickers[i]]['General'].pop('Listings', None)
        AddressData = raw_data[tickers[i]]['General'].pop('AddressData', None)
        NumberDividendsByYear = raw_data[tickers[i]]['SplitsDividends'].pop('NumberDividendsByYear', None)

        columns = ['General', 'Highlights', 'Valuation', 'SharesStats',
            'Technicals','SplitsDividends', 'AnalystRatings']
        for name in columns:
            single_ticker_clean_data[name] = pd.Series(raw_data[tickers[i]][name])

        columns = ['Institutions', 'Funds']
        for name in columns:
            single_ticker_clean_data[name] = pd.DataFrame(
                raw_data[tickers[i]]['Holders'][name]).T.set_index(['name'])
        single_ticker_clean_data['Holders'] = pd.concat([single_ticker_clean_data['Institutions'], single_ticker_clean_data['Funds']],
            keys=['Institutions','Funds'])
        Institutions = single_ticker_clean_data.pop('Institutions', None)
        Funds = single_ticker_clean_data.pop('Funds', None)

        single_ticker_clean_data['InsiderTransactions'] = pd.DataFrame(
            raw_data[tickers[i]]['InsiderTransactions']).T.set_index(['date'])

        columns = ['annual', 'quarterly']
        for name in columns:
            single_ticker_clean_data[name] = pd.DataFrame(
                raw_data[tickers[i]]['outstandingShares'][name]).T.set_index(['dateFormatted'])
        single_ticker_clean_data['outstandingShares'] = single_ticker_clean_data['quarterly'].join(
            single_ticker_clean_data['annual'], lsuffix='_quarterly', rsuffix='_annual')
        Institutions = single_ticker_clean_data.pop('quarterly', None)
        Funds = single_ticker_clean_data.pop('annual', None)

        # join on date
        columns = ['History', 'Trend', 'Annual']
        for name in columns:
            single_ticker_clean_data[name] = pd.DataFrame(raw_data[tickers[i]]['Earnings'][name]).T
        single_ticker_clean_data['Earnings'] = pd.concat([single_ticker_clean_data['History'], 
            single_ticker_clean_data['Trend'], single_ticker_clean_data['Annual']],
            keys=['History', 'Trend', 'Annual'], axis=1)
        History = single_ticker_clean_data.pop('History', None)
        Trend = single_ticker_clean_data.pop('Trend', None)
        Annual = single_ticker_clean_data.pop('Annual', None)

        statements = ['Balance_Sheet', 'Income_Statement', 'Cash_Flow']
        period = ['quarterly', 'yearly']

        for statement in statements:
            for time in period:
                # join on date
                single_ticker_clean_data[statement + time] = pd.DataFrame(
                    raw_data[tickers[i]]['Financials'][statement][time]).T

        # single_ticker_clean_data['Balance_Sheet'] = fundemental_data['Balance_Sheetquarterly'].join(
        #     fundemental_data['Balance_Sheetyearly'], lsuffix='_quarterly', rsuffix='_annual')
        single_ticker_clean_data['Balance_Sheet'] = pd.concat([single_ticker_clean_data['Balance_Sheetquarterly'], 
            single_ticker_clean_data['Balance_Sheetyearly']],
            keys=['Quarterly', 'Yearly'], axis=1)
        Balance_Sheetyearly = single_ticker_clean_data.pop('Balance_Sheetyearly', None)
        Balance_Sheetquarterly = single_ticker_clean_data.pop('Balance_Sheetquarterly', None)

        single_ticker_clean_data['Income_Statement'] = pd.concat([single_ticker_clean_data['Income_Statementquarterly'], 
            single_ticker_clean_data['Income_Statementyearly']],
            keys=['Quarterly', 'Yearly'], axis=1)
        Income_Statementyearly = single_ticker_clean_data.pop('Income_Statementyearly', None)
        Income_Statementquarterly = single_ticker_clean_data.pop('Income_Statementquarterly', None)

        
        single_ticker_clean_data['Cash_Flow'] = pd.concat([single_ticker_clean_data['Cash_Flowquarterly'], 
            single_ticker_clean_data['Cash_Flowyearly']],
            keys=['Quarterly', 'Yearly'], axis=1)
        Cash_Flowyearly = single_ticker_clean_data.pop('Cash_Flowyearly', None)
        Cash_Flowquarterly = single_ticker_clean_data.pop('Cash_Flowquarterly', None)

        all_clean_data[tickers[i]] = single_ticker_clean_data



    return all_clean_data

In [225]:
tickers = ['MSFT', 'AMZN']
stock_fundemental_data  = get_stock_fundmentals(tickers)

In [229]:
for ticker in tickers:
    print(ticker)
    print(stock_fundemental_data[ticker]['Highlights'].PERatio)
    #print(etf_fundemental_data[ticker]['Performance'])
    print('\n')

MSFT
29.7289


AMZN
102.11


